## DMCX 经济模型模拟包含的内容

- DMCX的释放模型，GWT兑换率变化
- DMC DAO的分红逻辑
- 公共数据挖矿的模型，包括 Sponosr的充值计算，矿工的质押计算，矿工的收益计算
- 公共数据挖矿的奖池分配模型（总金额，指定名次的奖励计算）
- 私有数据挖矿的模型，主要关注点也是矿工的收益计算和质押计算

## DMCX 的释放模型

从2024年8月开始一共有420个周期，每个周期大概是现实世界的1周。每21个周期进行一次难度调整。难度调整后的周期释放的Token是上一个周期的80%。总释放为5亿DMC。

第1-21周，每周最多释放480万枚DMC。随后的第21-42周，每周最多释放480万*0.8枚DMC

In [64]:
global exchange_cycle_data 
exchange_cycle_data = []
global unreleased_cycle_count 
unreleased_cycle_count = 0

def push_cycle_data(cycle_total):
    for i in range(21):
        cycle_data = {}
        cycle_data["exchange_gwt"] = 0
        cycle_data["ex_rate"] = 210
        cycle_data["dmcx_released"] = 0
        cycle_data['dmcx_balance'] = cycle_total
        exchange_cycle_data.append(cycle_data)

def print_dmx_release_table():
    n = 1
    count = 1
    for i in range(19):
        n = n * 0.8
        count = count + n
        

    init = ((50000*10000) / count) / 21
    print("first 1-21 cycle ,every cycle(week) relase dmcx:",init)
    push_cycle_data(init)

    token_count = init * 21
    balance = token_count
    cycle_index = 22
    for i in range (1,20):
        balance = balance * 0.8
        print(f"cycle {cycle_index} - {cycle_index+20} ,every cycle(week) release dmcx:",balance)
        push_cycle_data(balance)
        token_count += balance
        cycle_index += 21

    print("total release DMCX:",token_count)
    print("total not released all DMCX cycle count:",unreleased_cycle_count)

print_dmx_release_table()

first 1-21 cycle ,every cycle(week) relase dmcx: 4817446.134360656
cycle 22 - 42 ,every cycle(week) release dmcx: 80933095.05725902
cycle 43 - 63 ,every cycle(week) release dmcx: 64746476.04580722
cycle 64 - 84 ,every cycle(week) release dmcx: 51797180.83664578
cycle 85 - 105 ,every cycle(week) release dmcx: 41437744.66931663
cycle 106 - 126 ,every cycle(week) release dmcx: 33150195.735453304
cycle 127 - 147 ,every cycle(week) release dmcx: 26520156.588362645
cycle 148 - 168 ,every cycle(week) release dmcx: 21216125.270690117
cycle 169 - 189 ,every cycle(week) release dmcx: 16972900.216552094
cycle 190 - 210 ,every cycle(week) release dmcx: 13578320.173241675
cycle 211 - 231 ,every cycle(week) release dmcx: 10862656.13859334
cycle 232 - 252 ,every cycle(week) release dmcx: 8690124.910874672
cycle 253 - 273 ,every cycle(week) release dmcx: 6952099.928699738
cycle 274 - 294 ,every cycle(week) release dmcx: 5561679.942959791
cycle 295 - 315 ,every cycle(week) release dmcx: 4449343.9543678


### getCircleBalance@exchange.sol

```solidity
initial_dmc_balance = 4817446 ether;

function getCircleBalance(uint256 circle) public view returns (uint256) {
    //return 210 ether;

    uint256 adjust_times = (circle-1) / adjust_period;
    uint256 balance = initial_dmc_balance;
    for (uint i = 0; i < adjust_times; i++) {
        balance = balance * 4 / 5;
    }
    return balance;
}
```

## DMCX<->GWT的兑换

通过exchange.sol的接口，用户可以在一个周期里进行GWT<->DMCX的兑换。GWT和DMCX的兑换比率(ex_rate)是动态变化的.ex_rate每个周期开始时确定，在整个周期内不变。

基于该ex_rate:

- 可以将1000个GWT兑换成 ( 1000 / ex_rate) 个DMCX，兑换得到的DMCX来自本cycle待释放的DMCX（如果待释放DMCX消耗完了该调用会失败）,消耗的GWT会进入DMC DAO的分红池。
- 可以将1000个DMCX兑换成 ( 1000 * ex_rate * 1.1) 个GWT，该调用总是会成功,消耗的DMCX会被销毁。

### ex_rate的变化
ex_rate变化的基本逻辑是：
- 如果一个cycle里的DMCX都被兑换完了，那么下一个cycle的ex_rate就会增加,增加的比率和兑换完的时间有关，兑换的速度越快则ex_rate增加的越多，最多增加20%。
- 如果一个cycle里的DMCX没有兑换完，那么下一个cycle的ex_rate就会减少，剩余的未兑换DMCX越多则ex_rate下降的越多。ex_rate的下降最多为-20%。未兑换完成的DMCX会平滑的分配到未来的一些周期里，影响这些周期的可释放DMCX数量。



In [65]:

# if dmcx_released = 0,说明该周期的dmcx都释放了,used_time的值为0-7，说明是第几天挖完的

def set_dmcx_released(cycle_index, dmcx_released,used_time):
    global unreleased_cycle_count
    global exchange_cycle_data

    cycle_data= exchange_cycle_data[cycle_index - 1]
    next_cycle_data = exchange_cycle_data[cycle_index]

    
    if dmcx_released == 0:
        cycle_data["dmcx_released"] = cycle_data["dmcx_balance"]
        cycle_data["used_time"] = used_time
        new_rate = (7 - used_time) / 7
        if new_rate > 0.2:
            new_rate = 0.2
        next_cycle_data["ex_rate"] = cycle_data["ex_rate"] * (1+new_rate)
    else:
        cycle_data["dmcx_released"] = dmcx_released
        new_rate = dmcx_released / cycle_data["dmcx_balance"]
        if new_rate < 0.8:
            new_rate = 0.8
        next_cycle_data["ex_rate"] = cycle_data["ex_rate"] * new_rate
        addtion_dmcx = cycle_data["dmcx_balance"] - dmcx_released
        unreleased_cycle_count = unreleased_cycle_count + 1
        for i in range (unreleased_cycle_count):
            exchange_cycle_data[cycle_index + i]["dmcx_balance"] += (addtion_dmcx / unreleased_cycle_count)

    cycle_data["exchange_gwt"] = cycle_data["ex_rate"] * cycle_data["dmcx_released"]


def show_cycle_data(max_cycle):
    for i in range(max_cycle):
        cycle_data = exchange_cycle_data[i]
        if cycle_data["dmcx_released"] == cycle_data["dmcx_balance"]:
            print(f"all released cycle {i+1} dmcx_released:{cycle_data['dmcx_released']} exchange_gwt:{cycle_data['exchange_gwt']} ex_rate:{cycle_data['ex_rate']} dmcx_balance:{cycle_data['dmcx_balance']} used_time:{cycle_data['used_time']}")
        else:
            print(f"cycle {i+1} dmcx_released:{cycle_data['dmcx_released']} exchange_gwt:{cycle_data['exchange_gwt']} ex_rate:{cycle_data['ex_rate']} dmcx_balance:{cycle_data['dmcx_balance']}")
    
    return 


set_dmcx_released(1,0,6)
set_dmcx_released(2,0,5)
set_dmcx_released(3,0,3)
set_dmcx_released(4,0,3.5)
set_dmcx_released(5,0,6.3)
set_dmcx_released(6,4800000,0)
set_dmcx_released(7,2800000,0)
set_dmcx_released(8,2800000,0)
set_dmcx_released(9,0,6.5)
set_dmcx_released(10,0,6.9)

show_cycle_data(10)




all released cycle 1 dmcx_released:4817446.134360656 exchange_gwt:1011663688.2157378 ex_rate:210 dmcx_balance:4817446.134360656 used_time:6
all released cycle 2 dmcx_released:4817446.134360656 exchange_gwt:1156187072.2465575 ex_rate:240.0 dmcx_balance:4817446.134360656 used_time:5
all released cycle 3 dmcx_released:4817446.134360656 exchange_gwt:1387424486.695869 ex_rate:288.0 dmcx_balance:4817446.134360656 used_time:3
all released cycle 4 dmcx_released:4817446.134360656 exchange_gwt:1664909384.0350425 ex_rate:345.59999999999997 dmcx_balance:4817446.134360656 used_time:3.5
all released cycle 5 dmcx_released:4817446.134360656 exchange_gwt:1997891260.8420513 ex_rate:414.71999999999997 dmcx_balance:4817446.134360656 used_time:6.3
cycle 6 dmcx_released:4800000 exchange_gwt:2189721600.0 ex_rate:456.192 dmcx_balance:4817446.134360656
cycle 7 dmcx_released:2800000 exchange_gwt:1272711787.3241568 ex_rate:454.53992404434166 dmcx_balance:4834892.268721312
cycle 8 dmcx_released:2800000 exchange_g

### adjustExchangeRate@exchange.sol

```solidity
function adjustExchangeRate() internal {
    if(block.timestamp >= current_mine_circle_start + min_circle_time) {
        //end current cycle, calculate new exchange rate
        uint256 old_rate = dmc2gwt_rate;
        if(remain_dmc_balance > 0) {
            total_addtion_dmc_balance += remain_dmc_balance;
            addtion_circle_count += 1;

            // console.log("prev cycle dmc balance left %d, total left %d, total left cycle %d", remain_dmc_balance, total_addtion_dmc_balance, addtion_circle_count);

            //there has remaining DMCs in current cycle, decrease DMC -> GWT exchange rate
            dmc2gwt_rate = dmc2gwt_rate * (1-remain_dmc_balance/current_circle_dmc_balance);
            if (dmc2gwt_rate < old_rate * 4 / 5) {
                // we have a limit down as 20%
                dmc2gwt_rate = old_rate * 4 / 5;
            }
            if(dmc2gwt_rate < 210) {
                // the lowest rate is 210
                dmc2gwt_rate = 210;
            }
            // console.log("decrease dmc2gwt_rate to %d", dmc2gwt_rate);
        } else {
            if (addtion_circle_count > 0) {
                addtion_circle_count -= 1;
            }
            // all DMCs in current cycle have been mined, increase DMC -> GWT exchange rate
            dmc2gwt_rate = dmc2gwt_rate * (1+(current_finish_time-current_mine_circle_start)/min_circle_time);
            if(dmc2gwt_rate > old_rate * 6 / 5) {
                // we have a raising limit as 20%
                dmc2gwt_rate = old_rate * 6 / 5;
            }
            // for test
            // dmc2gwt_rate = 210;
            // console.log("increase dmc2gwt_rate to %d", dmc2gwt_rate);
        }

        emit gwtRateChanged(dmc2gwt_rate, old_rate);

        _newCycle();
    } else {
        // console.log("keep cycle.");
        require(remain_dmc_balance > 0, "no dmc balance in current circle");
    }
}
```

## 质押DMCX的分红奖励

DMCX的持有者可以将自己的DMCX质押到DMC DAO中，质押的DMCX会参与分红。分红的奖励来自于DMC DAO的分红池.其基本规则是：
- dividend_cycle: 分红周期，每次DMC DAO的分红池获得收入都会打入当前周期。获得收入时，还会检查当前时间与本周期启动时间的差，如果差大于3天，则开始一个新的分红周期，新的divident_cycle是当前cycle,并把上一个cycle标记为结束。
- 已结束的cycle里有确定的总收入和总质押DMCX数。
- 用户质押的DMCX，总是进入下一个分红周期（要等待下一个分红周期的开始）
- 用户可以从已结束的cycle里提取自己的分红奖励，提取的奖励是 (用户在该cycle质押的DMCX数 / 该cycle总质押DMCX数) * 该cycle总收入


In [66]:

global dividend_cycle_data 
dividend_cycle_data = []

def add_divident_cycle(total_stack,income):
    global dividend_cycle_data
    dividend_cycle_data.append({"dmcx":total_stack,"income":income})

def test_dividend(pelegement_cycle_index,end_cycle_index,dmcx_count):
    global dividend_cycle_data
    my_income = 0
    for i in range(pelegement_cycle_index+1,end_cycle_index+1):
        this_cycle_data = dividend_cycle_data[i]
        this_income = this_cycle_data["income"] * (dmcx_count / this_cycle_data["dmcx"])
        my_income += this_income
        print(f"pelgement {dmcx_count} dmcx at cycle:{i},income:{this_income}")

    print(f"pelgement {dmcx_count} dmcx from cycle:{pelegement_cycle_index} - {end_cycle_index} ,income:{my_income}")
    
add_divident_cycle(0,0)
add_divident_cycle(10000,1000)
add_divident_cycle(15000,2000)
add_divident_cycle(20000,3000)
add_divident_cycle(30000,4000)

test_dividend(1,3,5000)




pelgement 5000 dmcx at cycle:2,income:666.6666666666666
pelgement 5000 dmcx from cycle:1 - 3 ,income:666.6666666666666


### tryNewCycle@dividend.sol
    
```solidity
cycleMinLength = 3 days;
/**
    * Check if the new cycle should be started on check point
    * If the current cycle is over the max length, then start a new cycle
*/
function tryNewCycle() public {
    uint256 currentBlocktime = block.timestamp;
    
    CycleInfo storage currentCycle = cycles[currentCycleIndex];
    if (currentBlocktime - currentCycle.startBlocktime >= cycleMinLength) {
        currentCycleIndex = currentCycleIndex + 1;
        console.log("enter new cycle %d, totalStaked %d", currentCycleIndex, totalStaked);
        CycleInfo storage newCycle = cycles[currentCycleIndex];
        newCycle.startBlocktime = currentBlocktime;
        newCycle.totalStaked = totalStaked;
        
        if (currentCycle.totalStaked == 0) {
            newCycle.rewards = currentCycle.rewards;
        }

        emit NewCycle(currentCycleIndex, currentBlocktime);
    }
}
```